# ECSE 415: Final Project
Mathieu Geoffroy, 260986559
Ryan Reszetnik, 260948454


December 5th, 2023

In [23]:
import numpy as np
import cv2
import os
from ultralytics import YOLO
import torch
from collections import defaultdict
import matplotlib.pyplot as plt

working_dir = os.path.curdir

In [24]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model = YOLO('yolov8n.pt').to(device)

# import YOLO labels from the dataset
yolo_labels = model.names

In [25]:
def calc_flow(img1,img2):
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    return cv2.calcOpticalFlowFarneback(prev=img1_gray, 
                                    next=img2_gray, 
                                    flow=None,
                                    pyr_scale=0.5, 
                                    levels=3, 
                                    winsize=15,
                                    iterations=3, 
                                    poly_n=5, 
                                    poly_sigma=1.2, 
                                    flags=0)
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = img.copy()#cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (x2, y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis

In [26]:
def get_actual_speed(positions):
    new_pos = []
    #cap positions at 50 last positions
    if len(positions) > 50:
        positions = positions[-50:]
    for i in range(len(positions)):
        new_pos.append((positions[i][0].numpy(),positions[i][1].numpy()))
    new_pos = np.array(new_pos)
    speed = []
    for j in range(len(new_pos)-1):
        speed.append(np.sqrt((new_pos[j][0]-new_pos[j+1][0])**2+(new_pos[j][1]-new_pos[j+1][1])**2))
    speed = np.array(speed)
    new_pos = 1220-new_pos[:-1,0]
    speed = speed/(new_pos**2+abs(new_pos*speed)) * 1000000
    #median filter of size 3
    median_speed = []
    median_size = 5
    if len(speed) < median_size:
        return np.mean(speed)
    for i in range(len(speed)-median_size+1):
        median_speed.append(np.median(speed[i:i+median_size]))
    median_speed = np.array(median_speed)
    return np.mean(median_speed)


def is_moving(speed,predicted_speed):
    return speed > 1.5*predicted_speed

In [33]:
# Open the video file
def analyze_video(video_path, display=True, save=True):
    cap = cv2.VideoCapture(video_path)

    video_name = video_path.split('/')[-1].split('.')[0]
    
    out = cv2.VideoWriter(f'{video_name}_analyzed.mp4',cv2.VideoWriter_fourcc('m','p','4','v'), 30, (int(cap.get(3)),int(cap.get(4))))
    
    # Store the track history
    track_history = defaultdict(lambda: [])
    
    people_count = 0
    max_speed = 0
    car_moving = 0
    car_parked = 0

    predicted_speed = 30
    
    # Loop through the video frames
    frame_count = 0

    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()
        # flow = calc_flow(prev_frame, frame)
        # print(flow.shape)
        frame_count += 1
        # if frame_count == 90:
        #     success = False
        print("frame count", frame_count)
        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
 
            #disable the print statement in the track function
            results = model.track(frame, persist=True)
    
            # Get the boxes, masks, track IDs, class, for the frame
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            classes = results[0].boxes.cls.int().cpu().tolist()
    
            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            
            # Plot the tracks
            for box,track_id,cls in zip(boxes, track_ids,classes):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((x, y))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                if yolo_labels[cls] == 'person' and track_id not in track_history:
                    people_count += 1
                    

                obj_speed = get_actual_speed(track)
                if (yolo_labels[cls] == 'person' or yolo_labels[cls] == 'fire hydrant') and not np.isnan(obj_speed) and obj_speed < 100: 
                    predicted_speed = 29/30*predicted_speed + 1/30*obj_speed
                # set obj_speed to zero if it is Nan
                if np.isnan(obj_speed):
                    obj_speed = 0
                
                moving_pred = is_moving(obj_speed,predicted_speed)
                if obj_speed > 0 and not moving_pred and not np.isnan(obj_speed) and obj_speed < 100:
                    predicted_speed = 59/60*predicted_speed + 1/60*obj_speed
                
                text = "moving "+str(int(obj_speed)) if moving_pred else "stationary "+str(int(obj_speed))
                
                cv2.putText(annotated_frame, text, (int(x-w/2), int(y+h/2)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
               
                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

                if yolo_labels[cls] == 'car' and track_id not in track_history:
                    if moving_pred:
                        car_moving += 1
                    else:
                        car_parked += 1
  
            # Display the number of people
            cv2.putText(annotated_frame, f"Number of people: {people_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # Display the number of moving cars
            cv2.putText(annotated_frame, f"Number of moving cars: {car_moving}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            # Display the number of parked cars
            cv2.putText(annotated_frame, f"Number of parked cars: {car_parked}", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Display the predicted speed
            cv2.putText(annotated_frame, f"Speed: {predicted_speed} km/h", (50, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            # Display the annotated frame
            cv2.imshow(f'Tracking {video_name}', annotated_frame) if display else None

            # save the annotated frame to a new video
            out.write(annotated_frame) if save else None
    
            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
        else:
            # Break the loop if the end of the video is reached
            break
    
    # Release the video capture object and close the display window
    cap.release()
    out.release()
    cv2.destroyAllWindows()    

In [ ]:
#analyze_video(working_dir + '/mcgill_drive.mp4', display=True, save=True)
analyze_video(working_dir + '/st-catherines_drive.mp4', display=True, save=True)

frame count 1
0: 384x640 3 cars, 22.3ms
Speed: 8.7ms preprocess, 22.3ms inference, 10.2ms postprocess per image at shape (1, 3, 384, 640)
frame count 2
0: 384x640 3 cars, 16.0ms
Speed: 2.2ms preprocess, 16.0ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)
frame count 3
0: 384x640 3 cars, 15.1ms
Speed: 1.9ms preprocess, 15.1ms inference, 15.9ms postprocess per image at shape (1, 3, 384, 640)
frame count 4
0: 384x640 3 cars, 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 15.4ms postprocess per image at shape (1, 3, 384, 640)
frame count 5
0: 384x640 3 cars, 14.0ms
Speed: 2.1ms preprocess, 14.0ms inference, 16.3ms postprocess per image at shape (1, 3, 384, 640)
frame count 6
0: 384x640 3 cars, 15.1ms
Speed: 1.9ms preprocess, 15.1ms inference, 14.6ms postprocess per image at shape (1, 3, 384, 640)
frame count 7
0: 384x640 3 cars, 1 truck, 13.4ms
Speed: 1.8ms preprocess, 13.4ms inference, 16.3ms postprocess per image at shape (1, 3, 384, 640)
frame count 8
0: 384x640 4

In [ ]:





# moving_ids = [
#     3,5,6,7,10,11,19,30,31,42,45,46,50,63,64,69,75,94,106,119,121,122,135,141,147,149
# ]
# stopped_ids = [
#     1,2,25,83,87,91,92,95,109,112,133,150,153,155,156,154
# ]

all_ids = object_positions.keys()
all_speeds = [get_actual_speed(object_positions[i]) for i in object_positions.keys()]

CUTOFF = 0.05


print(all_speeds)
print("MOVING")
for i in range(len(all_speeds)):
    if all_speeds[i] > CUTOFF:
        print(all_ids[i],all_speeds[i])

print("STOPPED")
for i in range(len(all_speeds)):
    if all_speeds[i] <= CUTOFF:
        print(all_ids[i],all_speeds[i])


# moving_speeds = [get_actual_speed(object_positions[i]) for i in moving_ids]
# stopped_speeds = [get_actual_speed(object_positions[i]) for i in stopped_ids]

# #get average speed of moving objects
# moving_speeds = np.array(moving_speeds)
# #get average speed of stopped objects
# stopped_speeds = np.array(stopped_speeds)

# print(moving_speeds,stopped_speeds)

# #plot the speed of moving objects
# plt.plot(moving_speeds)
# plt.title("Speed of moving objects")
# plt.xlabel("Frame")
# plt.ylabel("Speed (km/h)")
# plt.show()

# #plot the speed of stopped objects
# plt.plot(stopped_speeds)
# plt.title("Speed of stopped objects")
# plt.xlabel("Frame")
# plt.ylabel("Speed (km/h)")
# plt.show()
# #0.09 cutoff


# print(max(stopped_speeds),min(moving_speeds))
# print(moving_speeds)
